<a href="https://colab.research.google.com/github/sumants-dev/CIS545-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandasql
!pip install pandas
!pip install requests
!pip install lxml
!pip install nltk

In [1]:
# Imports
import pandas as pd
import pandasql as psql
from lxml import html
import requests
import nltk

# Cleaning and Wrangling of Global Land Temperature Dataset
This dataset has the average monthly temperature for each country since 1743. For cleaning this dataset, we have two task:
    1. Convert alphabetic country names to 3-alpha iso codes
    2. Group by dates by year and country

## Extraction of country codes 
We extract the country name to country code mapping through wikipedia. Our approach is to make a request to the wikipedia page, and then use xpath to find the list of country names and country code.

In [2]:
def get_country_codes():
    wiki = requests.get("https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3")
    dom_tree = html.fromstring(wiki.content)
    xpath = "//div[@class=\"plainlist\"]/ul/li"
    country_list  = dom_tree.xpath(xpath)
    country_map = {}

    for country_elem in country_list:
        country_map[country_elem[2].text] = country_elem[1].text

    return country_map

        
def set_country_value(df):
    country_to_code = get_country_codes()
    df["Country"] = df["Country"].apply(lambda elem: country_to_code.get(elem))
    return df


In [3]:
global_temp_df = pd.read_csv('../Data/raw/GlobalLandTemperaturesByCountry.csv')
country_maps = get_country_codes()
global_temp_df = set_country_value(global_temp_df)
global_temp_df = global_temp_df.dropna()
global_temp_df

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
3239,1838-04-01,13.008,2.586,AFG
3241,1838-06-01,23.950,2.510,AFG
3242,1838-07-01,26.877,2.883,AFG
3243,1838-08-01,24.938,2.992,AFG
3244,1838-09-01,18.981,2.538,AFG
...,...,...,...,...
577456,2013-04-01,21.142,0.495,ZWE
577457,2013-05-01,19.059,1.022,ZWE
577458,2013-06-01,17.613,0.473,ZWE
577459,2013-07-01,17.000,0.453,ZWE


Now, we group by dates by year and country for the final result of global yearly temperature by country.

In [4]:
group_by_query = '''
SELECT strftime('%Y', dt) as Year, Country, AVG(AverageTemperature) as AvgYearlyTemp, AVG(AverageTemperatureUncertainty) as AvgTempUncertainty
FROM global_temp_df
GROUP BY strftime('%Y', dt), Country
'''

global_temps_final = psql.sqldf(group_by_query , locals())
global_temps_final.head()

,Year,Country,AvgYearlyTemp,AvgTempUncertainty
0,1743,ALB,8.620,2.268
1,1743,AND,7.556,2.188
2,1743,AUT,2.482,2.116
3,1743,BEL,7.106,1.855
4,1743,BGR,5.928,2.547


In [5]:
global_temps_final.to_csv('../Data/global_average_yearly_temp_clean.csv', index= False)

# Cleaning and Wrangling of Gas Emissions

In [6]:
gas_emissions_df = pd.read_csv('../Data/raw/owid-co2-data.csv')
gas_emissions_df = gas_emissions_df.drop(['country'], axis=1)
gas_emissions_df.head()

,iso_code,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [7]:
global_temps_final = pd.read_csv('../Data/global_average_yearly_temp_clean.csv')
global_temps_final.head(5)

,Year,Country,AvgYearlyTemp,AvgTempUncertainty
0,1743,ALB,8.620,2.268
1,1743,AND,7.556,2.188
2,1743,AUT,2.482,2.116
3,1743,BEL,7.106,1.855
4,1743,BGR,5.928,2.547


## Join the gas emission df with global temperature df

In [8]:
def find_columns_with_count(df, min_size = 10000):
    cols = df.columns
    cols_with_count = []

    for col in cols:
        count = df[col].count()

        if count > min_size:
            cols_with_count.append((col, count))

    cols_with_count.sort(key = lambda elem: elem[1])
    return cols_with_count
        

In [9]:
join_query = ''' SELECT gas.*, AvgYearlyTemp
FROM global_temps_final gt JOIN gas_emissions_df gas
ON gas.iso_code == gt.Country and gas.year == gt.Year
'''

join_df = psql.sqldf(join_query, locals())
cols_with_count = find_columns_with_count(join_df, 12500)
cols = [col[0] for col in cols_with_count]
join_df = join_df[cols].dropna()
join_df = join_df.set_index(['iso_code', 'year'])
join_df

oil_co2_per_capita  oil_co2  share_global_oil_co2  \
iso_code year                                                      
FRA      1855               0.001    0.037                100.00   
         1856               0.001    0.040                100.00   
         1857               0.001    0.044                 85.71   
         1858               0.001    0.048                 76.47   
         1859               0.001    0.048                 86.67   
...                           ...      ...                   ...   
WSM      2013               1.018    0.194                  0.00   
YEM      2013               0.848   21.321                  0.18   
ZAF      2013               0.922   49.501                  0.43   
ZMB      2013               0.218    3.254                  0.03   
ZWE      2013               0.307    4.104                  0.04   

               cumulative_oil_co2  share_global_cumulative_oil_co2  \
iso_code year                                                        
FRA      1855               0.037                           100.00   
         1856               0.077                           100.00   
         1857               0.121                            94.29   
         1858               0.169                            88.46   
         1859               0.216                            88.06   
...                           ...                              ...   
WSM      2013               5.141                             0.00   
YEM      2013             510.490                             0.10   
ZAF      2013            2010.813                             0.39   
ZMB      2013              88.452                             0.02   
ZWE      2013             122.709                             0.02   

               co2_growth_abs  co2_per_capita      co2  share_global_co2  \
iso_code year                                                              
FRA      1855           3.858           0.899   33.071             12.71   
         1856           1.561           0.940   34.632             12.49   
         1857           0.649           0.955   35.281             12.61   
         1858          -0.685           0.934   34.595             12.17   
         1859           0.396           0.943   34.991             11.61   
...                       ...             ...      ...               ...   
WSM      2013           0.007           1.018    0.194              0.00   
YEM      2013           6.708           0.993   24.976              0.07   
ZAF      2013          -5.273           8.491  455.866              1.29   
ZMB      2013           0.250           0.296    4.416              0.01   
ZWE      2013           3.953           0.887   11.836              0.03   

               cumulative_co2  share_global_cumulative_co2  co2_growth_prct  \
iso_code year                                                                 
FRA      1855         469.655                         7.96            13.21   
         1856         504.287                         8.17             4.72   
         1857         539.568                         8.36             1.87   
         1858         574.163                         8.52            -1.94   
         1859         609.155                         8.65             1.14   
...                       ...                          ...              ...   
WSM      2013           5.141                         0.00             3.92   
YEM      2013         537.482                         0.04            36.72   
ZAF      2013       17912.059                         1.24            -1.14   
ZMB      2013         204.904                         0.01             6.01   
ZWE      2013         701.598                         0.05            50.15   

               population  AvgYearlyTemp  
iso_code year                             
FRA      1855  36768948.0      11.763583  
         1856  36853256.0      12.615917  
         1857  36937755.0     

In [12]:
join_df.columns
join

Index(['oil_co2_per_capita', 'oil_co2', 'share_global_oil_co2',
       'cumulative_oil_co2', 'share_global_cumulative_oil_co2',
       'co2_growth_abs', 'co2_per_capita', 'co2', 'share_global_co2',
       'cumulative_co2', 'share_global_cumulative_co2', 'co2_growth_prct',
       'population', 'AvgYearlyTemp'],
      dtype='object')

In [ ]:
join_df.to_csv('../Data/global_average_yearly_temp_with_features_clean.csv')